# Use the `Knot` class to dockerize a python function and create a set of AWS resources

In [1]:
import logging
# Set the logging level to print logging statements in the notebook
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Change this to logging.DEBUG if you want more verbose output

In [2]:
# This example is housed in cloudknot/examples
# With cloudknot installed, you may simply import with
# `import cloudknot as ck`
# However, if you havent installed cloudknot yet
# (e.g. you're running this example from a fresh clone of the cloudknot github repo),
# you should temporarily add cloudknot to your sys.path using the commands below
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import cloudknot as ck

## Initialization

Create a test function, `test_func`, that `Knot` will dockerize. `Knot` uses `DockerImage` to Dockerize the input function. Refer to the `using_dockerimage` jupyter notebook example for more details on this.

In [3]:
def test_func(name=None, *, no_capitalize=False):
    """Test function for unit testing of cloudknot.DockerReqs

    Import statements of various formats are deliberately scattered
    throughout the function to test the pipreqs components of
    clouknot.DockerReqs
    """
    import AFQ
    import sys
    import boto3.ec2
    if name:
        from docker import api
        from os.path import join

        if not no_capitalize:
            import pytest as pt
            import h5py.utils as h5utils

            name = name.title()

        return 'Hello {0}!'.format(name)

    from six import binary_type as bt
    from dask.base import curry as dbc

    return 'Hello world!'

Create a `Knot` instance from the test function.

In [4]:
knot = ck.Knot(name='example', func=test_func)

INFO:cloudknot.dockerimage:Wrote Dockerfile /Users/Adam/code/projects/cloudknot/examples/cloudknot_docker_test_func_cpl6bv0w/Dockerfile
INFO:cloudknot.dockerimage:Building image cloudknot/test_func with tag cloudknot
INFO:cloudknot.cloudknot:knot example built docker image [{'tag': 'cloudknot', 'name': 'cloudknot/test_func'}]
INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): ecr.us-east-1.amazonaws.com
INFO:cloudknot.aws.ecr:Created repository cloudknot/test_func at 455598791984.dkr.ecr.us-east-1.amazonaws.com/cloudknot/test_func
INFO:cloudknot.cloudknot:knot example created/adopted docker repo cloudknot/test_func
INFO:cloudknot.dockerimage:Tagging image cloudknot/test_func with tag cloudknot
INFO:cloudknot.dockerimage:Pushing image cloudknot/test_func with tag cloudknot
INFO:cloudknot.cloudknot:knot example pushed it's docker image to the repo cloudknot/test_func
INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Starting n

And print the Amazon Resource Names of the created resources.

In [5]:
print(knot.pars.batch_service_role.arn)
print(knot.pars.ecs_instance_role.arn)
print(knot.pars.spot_fleet_role.arn)
print(knot.pars.vpc.vpc_id)
print(knot.pars.security_group.security_group_id)
print(knot.job_definition.arn)
print(knot.compute_environment.arn)
print(knot.job_queue.arn)
print(knot.docker_repo.repo_uri)

arn:aws:iam::455598791984:role/default-cloudknot-batch-service-role
arn:aws:iam::455598791984:role/default-cloudknot-ecs-instance-role
arn:aws:iam::455598791984:role/default-cloudknot-spot-fleet-role
vpc-6c0bf014
sg-7236d600
arn:aws:batch:us-east-1:455598791984:job-definition/example-cloudknot-job-definition:6
arn:aws:batch:us-east-1:455598791984:compute-environment/example-cloudknot-compute-environment
arn:aws:batch:us-east-1:455598791984:job-queue/example-cloudknot-job-queue
455598791984.dkr.ecr.us-east-1.amazonaws.com/cloudknot/test_func


In [6]:
retrieved_knot = ck.Knot(name='example')

INFO:cloudknot.cloudknot:Found knot example in config
INFO:cloudknot.cloudknot:Found PARS default in config
INFO:cloudknot.aws.iam:IAM role default-cloudknot-batch-service-role already exists: arn:aws:iam::455598791984:role/default-cloudknot-batch-service-role
INFO:cloudknot.cloudknot:PARS default adopted role default-cloudknot-batch-service-role
INFO:cloudknot.aws.iam:IAM role default-cloudknot-ecs-instance-role already exists: arn:aws:iam::455598791984:role/default-cloudknot-ecs-instance-role
INFO:cloudknot.cloudknot:PARS default adopted role default-cloudknot-ecs-instance-role
INFO:cloudknot.aws.iam:IAM role default-cloudknot-spot-fleet-role already exists: arn:aws:iam::455598791984:role/default-cloudknot-spot-fleet-role
INFO:cloudknot.cloudknot:PARS default adopted role default-cloudknot-spot-fleet-role
INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Resetting dropped connection: ec2.us-east-1.amazonaws.com
INFO:cloudknot.aws.ec2:VPC vpc-6c0bf014 already exists.
INF

In [7]:
knot.clobber(clobber_pars=True)

INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Resetting dropped connection: batch.us-east-1.amazonaws.com
INFO:cloudknot.aws.batch:Clobbered job queue example-cloudknot-job-queue
INFO:cloudknot.aws.batch:Clobbered compute environment example-cloudknot-compute-environment
INFO:cloudknot.aws.batch:Deregistered job definition example-cloudknot-job-definition
INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Resetting dropped connection: ecr.us-east-1.amazonaws.com
INFO:cloudknot.aws.ecr:Clobbered docker image cloudknot/test_func
INFO:cloudknot.dockerimage:Removed /Users/Adam/code/projects/cloudknot/examples/cloudknot_docker_test_func_cpl6bv0w/test_func.py
INFO:cloudknot.dockerimage:Removed /Users/Adam/code/projects/cloudknot/examples/cloudknot_docker_test_func_cpl6bv0w/Dockerfile
INFO:cloudknot.dockerimage:Removed /Users/Adam/code/projects/cloudknot/examples/cloudknot_docker_test_func_cpl6bv0w/requirements.txt
INFO:cloudknot.dockerimage:Removed /Users/Adam/